In [1]:
import pandas as pd

airline_df = pd.read_csv("airline_top20_df_filtered.csv")

In [2]:
airline_df.head()

,airline_name,content,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,value_money_rating,recommended
0,Air Canada,London to Toronto. Flight delayed 5 hrs and no...,Economy,3.0,1.0,1.0,1.0,1.0,no
1,Air Canada,We had a confirmed booking to fly from Toronto...,Economy,1.0,1.0,1.0,1.0,1.0,no
2,Air Canada,I travelled 27th July from Toronto to LHR. Thi...,Economy,1.0,1.0,4.0,2.0,1.0,no
3,Air Canada,Slow check-in in Beijing. Used the Air China B...,Business Class,5.0,2.0,2.0,2.0,3.0,yes
4,Air Canada,Indifferent cabin crew acted like passengers w...,Economy,3.0,3.0,2.0,1.0,2.0,no


#### Analiza sentimenta po ključnim aspektima aviokompanija, za sva tri dataset-a. Za svaku recenziju model predviđa ocenu za komfor sedišta, osoblje, hranu i odnos cene i kvaliteta itd. Rezultati se kombinuju po aviokompanijama i prikazuje se prosečna ocena za svaki aspekt.

In [3]:
from transformers import pipeline

sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment",
    truncation=True
)

def aspect_sentiment(text, aspects):
    results = {}
  
    for aspect in aspects:
        input_text = f"{aspect}: {text[:500]}" 
        res = sentiment_analyzer(input_text)[0]
        score_5 = int(res['label'][0])
        score_10 = 1 + (score_5 - 1) * 9 / 4
        results[aspect] = score_10
    return results


Device set to use cpu


In [4]:
aspects_airline = ["seat_comfort", "cabin_staff", "food_beverages", "value_money"]


all_results_airline = []

for idx, row in airline_df.iterrows():
    sentiment_scores = aspect_sentiment(row['content'], aspects_airline)
    sentiment_scores['airline_name'] = row['airline_name']
    all_results_airline.append(sentiment_scores)


airline = pd.DataFrame(all_results_airline)

summary_airline = (
    airline.groupby('airline_name')[aspects_airline]
    .mean()
    .round(2)  
    .reset_index()
)

print("Prosečne ocene ključnih aspekata aviokompanija:")
print(summary_airline)


Prosečne ocene ključnih aspekata aviokompanija:
                airline_name  seat_comfort  cabin_staff  food_beverages  \
0                 Air Canada          4.48         4.20            4.41   
1                 Air France          5.09         4.83            5.01   
2          American Airlines          3.94         3.60            3.87   
3            British Airways          5.31         5.12            5.27   
4     Cathay Pacific Airways          6.07         5.84            5.98   
5            Delta Air Lines          5.03         4.67            4.98   
6                   Emirates          5.74         5.52            5.71   
7             Etihad Airways          4.67         4.34            4.56   
8                Jet Airways          5.17         4.91            5.10   
9   Klm Royal Dutch Airlines          5.99         5.82            5.96   
10                 Lufthansa          6.17         5.94            6.11   
11         Malaysia Airlines          5.81         5

In [5]:
lounge_df = pd.read_csv("lounge_top20_df_filtered.csv")

In [6]:
aspects_lounge = ["comfort", "cleanliness", "bar_beverages", "catering", "washrooms", "wifi_connectivity", "staff_service"]

all_results_lounge = []
for idx, row in lounge_df.iterrows():
    sentiment_scores = aspect_sentiment(row['content'], aspects_lounge)
    sentiment_scores['airline_name'] = row['airline_name']
    all_results_lounge.append(sentiment_scores)

lounge = pd.DataFrame(all_results_lounge)

summary_lounge = (
    lounge.groupby('airline_name')[aspects_lounge]
    .mean()
    .round(2)  
    .reset_index()
)

print("\nProsečne ocene lounge-a po aviokompanijama:")
print(summary_lounge)



Prosečne ocene lounge-a po aviokompanijama:
                airline_name  comfort  cleanliness  bar_beverages  catering  \
0                 Air Canada     5.71         5.61           5.61      5.61   
1                 Air France     4.95         4.79           4.71      4.87   
2          American Airlines     4.67         4.48           4.51      4.54   
3            British Airways     5.52         5.37           5.35      5.41   
4     Cathay Pacific Airways     6.16         5.96           5.96      6.03   
5            Delta Air Lines     4.65         4.50           4.58      4.61   
6                   Emirates     6.14         6.03           6.05      6.03   
7             Etihad Airways     5.70         5.62           5.62      5.62   
8                Jet Airways     5.50         5.50           5.50      5.05   
9   Klm Royal Dutch Airlines     5.56         5.50           5.33      5.56   
10                 Lufthansa     5.64         5.59           5.52      5.54   
11     

In [7]:
seat_df = pd.read_csv("seat_top20_df_filtered.csv")

In [9]:
aspects_seat = ["seat_legroom", "seat_recline", "seat_width", "aisle_space", "viewing_tv"]

all_results_seat = []
for idx, row in seat_df.iterrows():
    sentiment_scores = aspect_sentiment(row['content'], aspects_seat)
    sentiment_scores['airline_name'] = row['airline_name']
    all_results_seat.append(sentiment_scores)

seat = pd.DataFrame(all_results_seat)

summary_seat = (
    seat.groupby('airline_name')[aspects_seat]
    .mean()
    .round(2)  
    .reset_index()
)

print("\nProsečne ocene sedišta po aviokompanijama:")
print(summary_seat)



Prosečne ocene sedišta po aviokompanijama:
                airline_name  seat_legroom  seat_recline  seat_width  \
0                 Air Canada          5.05          5.05        4.96   
1                 Air France          3.85          3.67        3.71   
2          American Airlines          3.20          3.15        3.20   
3            British Airways          4.35          4.24        4.24   
4     Cathay Pacific Airways          4.01          3.95        3.89   
5            Delta Air Lines          3.81          3.81        3.81   
6                   Emirates          3.76          3.73        3.73   
7             Etihad Airways          3.30          3.30        3.15   
8                Jet Airways          6.06          6.06        6.06   
9   Klm Royal Dutch Airlines          3.36          3.30        3.36   
10                 Lufthansa          4.24          4.20        4.24   
11         Malaysia Airlines          5.39          5.16        5.28   
12            Qantas

#### Upis podataka u bazu, za veće skupove podataka treba promeniti način upisa!

In [10]:
import psycopg2

conn = None
cursor = None

try:
    conn = psycopg2.connect(
        host="localhost",
        database="airline_recommendations_db",
        user="postgres",
        password="postgres",
        port=5432
    )
    cursor = conn.cursor()

    cursor.execute("TRUNCATE TABLE seat_sentiment_summary;")

    for _, row in summary_seat.iterrows():
        cursor.execute(
            """
            INSERT INTO seat_sentiment_summary (
                airline_name,
                seat_legroom,
                seat_recline,
                seat_width,
                aisle_space,
                viewing_tv
            )
            VALUES (%s, %s, %s, %s, %s, %s);
            """,
            (
                row['airline_name'],
                float(row['seat_legroom']),
                float(row['seat_recline']),
                float(row['seat_width']),
                float(row['aisle_space']),
                float(row['viewing_tv'])
            )
        )

    conn.commit()
    print("Seat podaci uspešno upisani!")

except Exception as e:
    print("Greška prilikom upisa seat_sentiment_summary:", e)

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Seat podaci uspešno upisani!


In [11]:
import psycopg2

conn = None
cursor = None

try:
    conn = psycopg2.connect(
        host="localhost",
        database="airline_recommendations_db",
        user="postgres",
        password="postgres",
        port=5432
    )
    cursor = conn.cursor()

    cursor.execute("TRUNCATE TABLE lounge_sentiment_summary;")

    for _, row in summary_lounge.iterrows():
        cursor.execute(
            """
            INSERT INTO lounge_sentiment_summary (
                airline_name,
                comfort,
                cleanliness,
                bar_beverages,
                catering,
                washrooms,
                wifi_connectivity,
                staff_service
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
            """,
            (
                row['airline_name'],
                float(row['comfort']),
                float(row['cleanliness']),
                float(row['bar_beverages']),
                float(row['catering']),
                float(row['washrooms']),
                float(row['wifi_connectivity']),
                float(row['staff_service'])
            )
        )

    conn.commit()
    print("Lounge podaci uspešno upisani!")

except Exception as e:
    print("Greška prilikom upisa lounge_sentiment_summary:", e)

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Lounge podaci uspešno upisani!


In [12]:
import psycopg2

conn = None
cursor = None

try:
    conn = psycopg2.connect(
        host="localhost",
        database="airline_recommendations_db",
        user="postgres",
        password="postgres",
        port=5432
    )
    cursor = conn.cursor()

    cursor.execute("TRUNCATE TABLE airline_sentiment_summary;")

    for _, row in summary_airline.iterrows():
        cursor.execute(
            """
            INSERT INTO airline_sentiment_summary (
                airline_name,
                seat_comfort,
                cabin_staff,
                food_beverages,
                value_money
            )
            VALUES (%s, %s, %s, %s, %s);
            """,
            (
                row['airline_name'],
                float(row['seat_comfort']),
                float(row['cabin_staff']),
                float(row['food_beverages']),
                float(row['value_money'])
            )
        )

    conn.commit()
    print("Airline podaci uspešno upisani!")

except Exception as e:
    print("Greška prilikom upisa airline_sentiment_summary:", e)

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Airline podaci uspešno upisani!
